In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
urls = []
target_addresses = []
with open('./input_hw1.txt', 'r') as file:
    for target_address in file.read().splitlines():
        target_addresses.append(target_address)
        urls.append("https://www.blockchain.com/eth/address/" + target_address + "?view=standard")

In [3]:
def get_info(url):
    html = requests.get(url)
    bs = BeautifulSoup(html.text, 'html.parser')
    
    # basic information
    infos = bs.find_all("div", class_="sc-8sty72-0 cyLejs")
    info = []
    for i in range(2, len(infos), 2):
        info.append(infos[i].text + ': ' + infos[i+1].text)
        
    # withdraw information
    dates = bs.find_all("div", class_="sc-1rk8jst-0 jKHeRh")
    tos = bs.find_all("div", class_="sc-1rk8jst-2 jyQjOo")
    amounts = bs.find_all("div", class_="sc-1rk8jst-1 jyQjOn")

    date = 0
    next_address = 0
    next_url = 0
    
    for i in range(len(amounts)-1, -1, -1):
        if amounts[i].text[0] == '-':
            date = dates[i].text
            to = tos[i].text
            amount = amounts[i].text
            break

    if date != 0:
        info.append('Date' + ': ' + date)
        info.append('To' + ': ' + to)
        info.append('Amount' + ': ' + amount)

        next_address = to
        next_url = "https://www.blockchain.com/eth/address/" + next_address + "?view=standard"
        
    return info, next_address, next_url

In [4]:
all_info = []
for i in range(len(urls)):
    addresses = []
    addresses.append(target_addresses[i])
    
    info, next_address, next_url = get_info(urls[i])
    all_info.append(info)
    addresses.append(next_address)
    
    count = 0
    while next_address != 0 and count < 3:
        next_info, next_address, next_url = get_info(next_url)
        all_info.append(next_info)
        count += 1
        
        if next_address != 0 and count < 3:
            addresses.append(next_address)
            
    all_info.append(addresses)

In [5]:
output = []
for i in range(len(all_info)):
    if all_info[i][0][:5] == 'Nonce':
        for j in all_info[i]:
            output.append(j)
        output.append('--------------------------------------------------------------------------')
    else:
        string = all_info[i][0]
        if len(all_info[i]) > 1:
            for k in range(len(all_info[i])-1):
                string += ' -> ' + all_info[i][k+1]
        output.append(string)
        output.append('--------------------------------------------------------------------------')

In [6]:
with open("108065530_hw1_output.txt", mode='wt', encoding='utf-8') as out:
    out.write('\n'.join(output))